# 数据规整化：清理、转换、合并和重塑

## 合并数据集

pandas 的 merge 函数是核心

In [1]:
from pandas import DataFrame
df1 = DataFrame({
    'key':list('bbacaab'),
    'data1':range(7)
})
df2 = DataFrame({
    'key':['a','b','d'],
    'data2':range(3)
})


In [2]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [3]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [4]:
import pandas as pd
pd.merge(df1,df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


这里并没有指定将那个列进行连接，如果没有指定，就会将重叠列的列名当做键。
一般我们会显式地指定。

In [5]:
pd.merge(df1,df2,on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


如果两个对象的列名不同，也可以分别进行指定

In [7]:
df3 = DataFrame({
    'lkey':list('bbacaab'),
    'data1':range(7)
})
df4 = DataFrame({
    'rkey':['a','b','d'],
    'data2':range(3)
})

In [8]:
pd.merge(df3,df4,left_on = 'lkey',right_on = 'rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


注意到 结果中的 c d 和相关的数据均消失了，默认情况下,merge做的是内连接，结果中的键是**交集**

下面给一个外连接的例子

In [9]:
pd.merge(df1,df2,how="outer")

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


多对多连接产生的是行的笛卡尔积，连接方式只会影响结果中的键

## 索引上的合并
如果 DataFrame 的连接键位于其索引中，可以传入

left_index = True 或者 right_index = True

## 轴向连接

另外一种数据合并运算时堆叠，其一个例子是用于合并 Numpy 数组的 concatenation 函数

In [10]:
import numpy as np
arr = np.arange(12).reshape((3,4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [11]:
np.concatenate([arr,arr],axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

对于 pandas 而言，其中的 concat 函数提供了一种方式

In [13]:
from pandas import Series
s1 = Series([0,1],index = ['a','b'])
s2 = Series([2,3,4],index = ['c','d','e'])
s3 = Series([5,6],index = ['f','g'])
pd.concat([s1,s2,s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

默认的时候会产生一个新的 Series

也可以传入 axis = 1 产生一个新的 DataFrame

In [15]:
pd.concat([s1,s2,s3],axis=1)

D:\Software\Code\Conda\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


可以指定在其他轴上使用的索引

In [19]:
s4 = pd.concat([s1*5,s3])
pd.concat([s1,s4],axis =1 ,join_axes=[['a','c','b','e']])

,0,1
a,0.0,0.0
c,NaN,NaN
b,1.0,5.0
e,NaN,NaN


还可以在连接轴上创建一个层次化索引，用于区分参与连接的片段的结果

In [20]:
result = pd.concat([s1,s2,s3],keys = ['one','two','three'])
result

one    a    0
       b    1
two    c    2
       d    3
       e    4
three  f    5
       g    6
dtype: int64

In [21]:
result.unstack()

,a,b,c,d,e,f,g
one,0.0,1.0,NaN,NaN,NaN,NaN,NaN
two,NaN,NaN,2.0,3.0,4.0,NaN,NaN
three,NaN,NaN,NaN,NaN,NaN,5.0,6.0


In [22]:
pd.concat([s1,s2,s3],axis=1,keys = ['one','two','three'])

D:\Software\Code\Conda\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


此时 keys 是 DataFrame 的列头

另外如果传入的是一个字典，那么字典的键就会被当做是 keys 选项的值

In [23]:
pd.concat({
    'level1':df1,
    "level2":df2
},axis=1)

level1       level2      
     key data1    key data2
0      b     0      a   0.0
1      b     1      b   1.0
2      a     2      d   2.0
3      c     3    NaN   NaN
4      a     4    NaN   NaN
5      a     5    NaN   NaN
6      b     6    NaN   NaN

另外，还可以忽略行索引


In [24]:
df1 = DataFrame(np.random.randn(3,4),columns=list('abcd'))
df2 = DataFrame(np.random.randn(2,3),columns=list('bda'))
df1

,a,b,c,d
0,-0.154313,-0.530861,-0.431613,-0.918940
1,2.235435,0.640033,0.964853,0.251914
2,0.177035,0.418284,-0.831784,1.014284


In [25]:
df2

,b,d,a
0,0.946702,-0.359893,0.289705
1,-1.314504,-1.229710,0.836599


In [26]:
pd.concat([df1,df2],ignore_index = True)

D:\Software\Code\Conda\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


,a,b,c,d
0,-0.154313,-0.530861,-0.431613,-0.918940
1,2.235435,0.640033,0.964853,0.251914
2,0.177035,0.418284,-0.831784,1.014284
3,0.289705,0.946702,NaN,-0.359893
4,0.836599,-1.314504,NaN,-1.229710


## 合并重叠数据

有一些情况下不能使用简单的合并或者连接运算进行处理，比如有索引全部或者部分
重叠的两个数据集

In [27]:
a = Series([np.nan,2.3,np.nan,3.5,4.7,np.nan],
          index = list('fedcba'))
b = Series(np.arange(len(a),dtype = np.float64),
          index = list('fedcba'))
b[-1] = np.nan
a

f    NaN
e    2.3
d    NaN
c    3.5
b    4.7
a    NaN
dtype: float64

In [28]:
b


f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

In [29]:
np.where(pd.isnull(a),b,a)

array([0. , 2.3, 2. , 3.5, 4.7, nan])

可以使用 combine_first 进行数据补齐

In [30]:
b[:-2].combine_first(a[2:])

a    NaN
b    4.7
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

## 重塑和轴向旋转
**重塑层次化索引**

+ stack: 将数据的列“旋转”为行
+ unstack:将数据的行“旋转”为列

In [32]:
data = DataFrame(np.arange(6).reshape((2,3)),
                index = pd.Index(['Ohio','Colorado'],
                name = "state"),
                columns = pd.Index(['one','two','three'],name = 'number'))

In [33]:
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [35]:
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [36]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [37]:
df = DataFrame({'left':result,'right':result+5},
              columns=pd.Index(['left','right'],name = 'side'))
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [38]:
df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

## 数据转换
### 移除重复数据

In [39]:
data = DataFrame({
    'k1':['one']*3+['two']*4,
    'k2':[1,1,2,3,3,4,4]
})
data

,k1,k2
0,one,1
1,one,1
2,one,2
3,two,3
4,two,3
5,two,4
6,two,4


In [40]:
data.duplicated()

0    False
1     True
2    False
3    False
4     True
5    False
6     True
dtype: bool

返回一个布尔型的 Series ，表示各行是否为重复行

In [41]:
data.drop_duplicates()

,k1,k2
0,one,1
2,one,2
3,two,3
5,two,4


In [42]:
data['v1'] = range(7)
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
3,two,3,3


### 利用函数或者映射进行数据转换

Series 的 map 方法可以接受一个函数或者含有映射关系的字典型对象

使用 map 是一种实现元素级转换的方式

### 替换值
replace 相比 map 更加灵活和简单

In [43]:
data = Series([1.,-999.,2.,-999.,-1000.,3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [44]:
data.replace(-999,np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [45]:
data.replace([-999,-1000],[np.nan,0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [46]:
data.replace({-999:np.nan,-1000:0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### 重命名轴索引

In [48]:
data = DataFrame(np.arange(12).reshape((3,4)),
                index = ['ohio','Colorado','New York'],
                columns = ['one','two','three','four'])

In [49]:
data.index.map(str.upper)

Index(['OHIO', 'COLORADO', 'NEW YORK'], dtype='object')

可以直接赋值

或者也可以使用 rename 得到复制版本

In [50]:
data.index = data.index.map(str.upper)
data

,one,two,three,four
OHIO,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


In [51]:
data.rename(index = str.title,columns = str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


rename 可以结合字典对于部分轴标签进行更新

In [52]:
data.rename(index = {'OHIO':'INDIANA'},
           columns = {'three':'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


In [53]:
_ = data.rename(index = {'OHIO':'INDIANA'},inplace = True)
data

,one,two,three,four
INDIANA,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


### 离散化和面元划分

In [54]:
ages = [20,22,25,27,21,23,37,31,61,45,41,32]
bins = [18,25,35,60,100]
cats = pd.cut(ages,bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [59]:
pd.value_counts(cats)

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

可以用 right = False 修改闭端

In [60]:
group_names = ['Youth','YoungAdult','MiddleAged','Senior']
pd.cut(ages,bins,labels = group_names)

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

也可以传入面元的数量进行等长划分

In [61]:
data = np.random.rand(20)
pd.cut(data,4,precision = 2)

[(0.31, 0.52], (0.31, 0.52], (0.31, 0.52], (0.73, 0.95], (0.31, 0.52], ..., (0.1, 0.31], (0.1, 0.31], (0.73, 0.95], (0.52, 0.73], (0.1, 0.31]]
Length: 20
Categories (4, interval[float64]): [(0.1, 0.31] < (0.31, 0.52] < (0.52, 0.73] < (0.73, 0.95]]

还有 qcut 函数，可以更加**样本分位数**进行划分

In [62]:
data = np.random.randn(1000)
cats = pd.qcut(data,4) # 按照四分位数进行切割
cats

[(-3.593, -0.648], (0.707, 3.756], (-3.593, -0.648], (0.707, 3.756], (0.0316, 0.707], ..., (-0.648, 0.0316], (-0.648, 0.0316], (0.0316, 0.707], (-3.593, -0.648], (0.0316, 0.707]]
Length: 1000
Categories (4, interval[float64]): [(-3.593, -0.648] < (-0.648, 0.0316] < (0.0316, 0.707] < (0.707, 3.756]]

In [63]:
pd.value_counts(cats)

(0.707, 3.756]      250
(0.0316, 0.707]     250
(-0.648, 0.0316]    250
(-3.593, -0.648]    250
dtype: int64

In [64]:
pd.qcut(data,[0,0.1,0.5,0.9,1.])

[(-1.289, 0.0316], (1.234, 3.756], (-1.289, 0.0316], (0.0316, 1.234], (0.0316, 1.234], ..., (-1.289, 0.0316], (-1.289, 0.0316], (0.0316, 1.234], (-1.289, 0.0316], (0.0316, 1.234]]
Length: 1000
Categories (4, interval[float64]): [(-3.593, -1.289] < (-1.289, 0.0316] < (0.0316, 1.234] < (1.234, 3.756]]

这两个离散化函数对于分量和分组分析十分重要
### 检测和过滤异常值

In [65]:
np.random.seed(12345)
data = DataFrame(np.random.randn(1000,4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.067684,0.067924,0.025598,-0.002298
std,0.998035,0.992106,1.006835,0.996794
min,-3.428254,-3.548824,-3.184377,-3.745356
25%,-0.774890,-0.591841,-0.641675,-0.644144
50%,-0.116401,0.101143,0.002073,-0.013611
75%,0.616366,0.780282,0.680391,0.654328
max,3.366626,2.653656,3.260383,3.927528


In [66]:
col = data[3]
col[np.abs(col) > 3]

97     3.927528
305   -3.399312
400   -3.745356
Name: 3, dtype: float64

In [68]:
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
5,-0.539741,0.476985,3.248944,-1.021228
97,-0.774363,0.552936,0.106061,3.927528
102,-0.655054,-0.565230,3.176873,0.959533
305,-2.315555,0.457246,-0.025907,-3.399312
324,0.050188,1.951312,3.260383,0.963301
400,0.146326,0.508391,-0.196713,-3.745356
499,-0.293333,-0.242459,-3.056990,1.918403
523,-3.428254,-0.296336,-0.439938,-0.867165
586,0.275144,1.179227,-3.184377,1.369891
808,-0.362528,-3.548824,1.553205,-2.186301


### 排列和随机取样


In [69]:
df = DataFrame(np.arange(5*4).reshape((5,4)))
sampler = np.random.permutation(5)
sampler

array([1, 0, 2, 3, 4])

In [70]:
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [71]:
df.take(sampler)

,0,1,2,3
1,4,5,6,7
0,0,1,2,3
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [72]:
df.take(np.random.permutation(len(df))[:3])

,0,1,2,3
1,4,5,6,7
3,12,13,14,15
4,16,17,18,19


上面是不用替换的方法进行随机采样，如果使用替换的方式产生样本，那么

In [74]:
bag = np.array([5,-7,-1,6,4])
sampler = np.random.randint(0,len(bag),size = 10)
sampler

array([4, 4, 2, 2, 2, 0, 3, 0, 4, 1])

In [76]:
draws = bag.take(sampler)
draws

array([ 4,  4, -1, -1, -1,  5,  6,  5,  4, -7])

### 计算指标/哑变量

在机器学习中常常会将分类变量转换为指标矩阵

In [79]:
df = DataFrame({'key':list('bbacab'),
              'data1':range(6)})
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [80]:
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


## 字符串操作

大部分的文本运算有着内置的字符串对象的内置方法，对于更加复杂的模式匹配和文本操作，则可能用到正则表达式

In [81]:
val = 'a,b,   guido'
val.split(',')

['a', 'b', '   guido']

In [83]:
pieces = [x.strip() for x in val.split(',')]

split 常常 结合 strip(用于修剪空白符（包括换行）)来使用

In [84]:
first , second, third = pieces
first + "::" + second + "::" + third

'a::b::guido'

另外一种更加常用的方法是

In [85]:
'::'.join(pieces)

'a::b::guido'

对于子串定位最佳的是使用 in 

当然可以使用 index 和 find


In [86]:
'guido' in val

True

In [87]:
val.index(',')

1

In [88]:
val.find(':')

-1

注意：find 和 index 的区别在于，如果找不到字符串，index 将会引发一个异常，而不是返回-1

In [89]:
val.index(':')

ValueError: substring not found

In [91]:
val.find(":")

-1

count 可以返回指定子串出现次数

In [92]:
val.count(',')

2

replace 可以将指定模式替换为另外一个模式，也常常用于删除：传入一个空字符串

In [93]:
val.replace(',','::')

'a::b::   guido'

In [94]:
val.replace(',','')

'ab   guido'

### 正则表达式

正则表达式提供了一种方便地在文本中搜索或者匹配字符串模式的方式

In [95]:
from IPython.display import IFrame
IFrame('https://www.princeton.edu/~mlovett/reference/Regular-Expressions.pdf', width=800, height=450)

In [97]:
IFrame('https://www.wikiwand.com/en/Regular_expression', width=800, height=450)

### Pandas 中矢量化的字符串函数